# Calculadora de Tempo Perdido

## 1. Ideia do Projeto
Script em Python que acessa os dados de tempo de uso do sistema operacional.

Calcula quanto tempo o usuário gasta em atividades como:
- Redes sociais / Navegadores
- Streaming (filmes, séries, YouTube)
- Jogos
- Aplicativos de produtividade ou outros

Converte esses dados para estatísticas semanais, mensais e anuais.
Mostra equivalentes criativos, como livros lidos, filmes assistidos ou km corridos.

## 2. Ferramentas:
- Python 3.12
    - Windows: leitura de registros do Tempo de Uso / Focus Assist via winreg ou API de produtividade do Windows (possivelmente via pywin32).
    - macOS: via AppleScript (osascript) ou frameworks como CoreData/ScreenTime.
    - Linux: pode depender do ambiente desktop (GNOME/KDE) ou logs de atividade.
- tabulate / prettytable → para tabelas no terminal.
- matplotlib → gráficos de tempo perdido.

## 3. Fases do projeto
- Coleta de dados
    - Acessar os dados nativos de tempo de uso do SO.
    - Mapear aplicativos e categorizar em grupos (ex: Chrome → Redes Sociais/Streaming).

- Calculos principais
    - Converter o tempo em horas por semana/mês/ano
    - Transformar em equivalentes de produtividade: livros, filmes, maratonas, km, etc.

- Exibir resultados de forma clara e intuitiva (tabela).

- Visual extra
    - Gráficos mostrando tempo por categoria.
    - Relatórios exportáveis em .txt ou .pdf.

In [ ]:
# Lib para coletar informações do SO windows
import wmi

In [77]:
# Lib para manipular dados e tabelas
import pandas as pd

# Converter horários de logs para OBJ de tempo, subtrair hora de fechamento-hora de abertura=tempo gasto, agrupar os dados por dia da semana
from datetime import datetime, timedelta

In [ ]:
# Nomes de aplicativos para ignorar
ignore_list = [
    "svchost.exe", "RuntimeBroker.exe", "dllhost.exe",
    "SearchIndexer.exe", "conhost.exe", "LockApp.exe",
    "wlanext.exe", "unsecapp.exe", "WmiPrvSE.exe", "lsass.exe",
    "csrss.exe", "smss.exe", "wininit.exe", "winlogon.exe",
    "System Idle Process", "explorer.exe", "py.exe", "python.exe", "pythonw.exe"
]

In [78]:
# Mapeamento de categorias de apps
categories = {
    "chrome.exe": "Navegador",
    "msedge.exe": "Navegador",
    "Discord.exe": "Comunicação",
    "WhatsApp.exe": "Comunicação",
    "steam.exe": "Jogos",
    "steamwebhelper.exe": "Jogos",
    "pycharm64.exe": "Produtividade",
    "powershell.exe": "Produtividade",
    "idea64.exe": "Produtividade",
    "code.exe": "Produtividade",
    "Spotify.exe": "Entretenimento",
    "vlc.exe": "Entretenimento",
    "Netflix.exe": "Entretenimento"
}

In [ ]:
# --- Analisador de Dados Históricos ---
def run_analyzer_historical():
    try:
        # Conectar ao WMI
        conn = wmi.WMI()

        # Definir período: últimos 7 dias
        data_limite = datetime.now() - timedelta(days=7)

        # Lista para armazenar dados coletados
        processos = []

        # Obter todos os processos criados na última semana
        # Nota: WMI não armazena a hora de término. Para os processos que estão ativos, a duração
        # é calculada até o momento da execução do script. Para os que foram encerrados,
        # a duração exata não é conhecida, por isso esta abordagem é uma estimativa.
        for p in conn.Win32_Process():
            try:
                start_time_str = p.CreationDate.split('.')[0]
                start_time = datetime.strptime(start_time_str, "%Y%m%d%H%M%S")

                if start_time >= data_limite:
                    # Para simplificar e manter a lógica do seu código, a duração é calculada
                    # do início até agora. Isso é uma estimativa para o propósito do projeto.
                    duration = (datetime.now() - start_time).total_seconds() / 3600  # horas

                    processos.append({
                        "app": p.Name,
                        "start_time": start_time,
                        "duration_hours": duration
                    })
            except Exception as e:
                # Ignora erros em processos do sistema que não têm data de criação
                continue

In [79]:
# Lista para armazenar dados coletados
processos = []

In [80]:
# Iterar pelos processos recentes
for p in conn.Win32_Process():
    try:
        # Pega informações de criação
        start_time = datetime.strptime(p.CreationDate.split('.')[0], "%Y%m%d%H%M%S")
        if start_time >= data_limite:
            # Se não tiver fim registrado, ele considera que terminou agora
            end_time = datetime.now()
            processos.append({
                "app": p.Name,
                "start_time": start_time,
                "end_time": end_time,
                "duration": (end_time - start_time).total_seconds()/3600  # horas
            })
    except:
        continue

In [81]:
# Criar DataFrame
df = pd.DataFrame(processos)

In [82]:
ignore_list = [
    "svchost.exe", "RuntimeBroker.exe", "dllhost.exe",
    "SearchIndexer.exe", "conhost.exe", "LockApp.exe",
    "wlanext.exe", "unsecapp.exe", "WmiPrvSE.exe"
]
df = df[~df["app"].isin(ignore_list)]

In [83]:
categories = {
    "chrome.exe": "Navegador",
    "msedge.exe": "Navegador",
    "Discord.exe": "Comunicação",
    "WhatsApp.exe": "Comunicação",
    "steam.exe": "Jogos",
    "steamwebhelper.exe": "Jogos",
    "pycharm64.exe": "Produtividade",
    "powershell.exe": "Produtividade",
}
df["category"] = df["app"].map(categories).fillna("Outros")

In [84]:
# Agrupar por aplicativo
resumo = df.groupby("category")["duration"].sum().sort_values(ascending=False)

In [85]:
print(resumo)

category
Outros           5597.840119
Comunicação       359.700773
Navegador         229.749683
Jogos              54.705138
Produtividade       8.047676
Name: duration, dtype: float64


In [86]:

def converter_tempo_flexivel(valores, unidade_preferida=None):
    """
    Converte os tempos de uso para minutos, horas ou dias.

    parâmetros:
    - valores: pd.Series com tempo em minutos
    - unidade_preferida: "minutos", "horas" ou "dias". Se None, a função escolhe automaticamente

    retorna:
    - valores convertidos
    - unidade usada
    """
    total_minutos = valores.sum()

    # Se o usuário não especificou, escolhe automaticamente
    if unidade_preferida is None:
        if total_minutos < 60:
            unidade = "minutos"
        elif total_minutos < 24 * 60:
            unidade = "horas"
        else:
            unidade = "dias"
    else:
        unidade = unidade_preferida

    # Converte
    if unidade == "minutos":
        return valores, "minutos"
    elif unidade == "horas":
        return valores / 60, "horas"
    elif unidade == "dias":
        return valores / (60*24), "dias"
    else:
        raise ValueError("Unidade inválida! Escolha minutos, horas ou dias.")

# -------------------------------
# Exemplo de uso

convertidos, unidade = converter_tempo_flexivel(resumo)
print(convertidos)
print(f"Unidade usada: {"minutos"}")

category
Outros           3.887389
Comunicação      0.249792
Navegador        0.159548
Jogos            0.037990
Produtividade    0.005589
Name: duration, dtype: float64
Unidade usada: minutos
